In [1]:
from ortools.linear_solver import pywraplp
#from ortools.sat.python import cp_model
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#def solveSubproblem(z, ygh, yij, y, xp, E, Lamda1, Lamda2, solver):
def solveSubproblem(Lambda1, Lambda2, z):    
    # The constrains need to be relaxed
    z3 = sum(Lambda1[i][j][g][h][k]*(ygh[i][j][g][h][k]-yij[i][j][g][h][k]-y[i][j][g][h][k]*E[i][j])
        for i in range(P)
        for j in range(T)
        for g in range(P)
        for h in range(T)
        for k in range(R)
    )

    z4 = sum(Lambda2[i][j][g][h][k]*(y[i][j][g][h][k]+y[g][h][i][j][k]-xp[g][h][i][j][k])
        for i in range(P)
        for j in range(T)
        for g in range(P)
        for h in range(T)
        for k in range(R)
    )


    z = z + z3 - z4

    # Solve the problem.
    print(f'Solving with {solver.SolverVersion()}')

    objective = z
    solver.Minimize(objective)
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Weight:', w, 'Solution:', solver.Objective().Value())
    else:
        print('No feasible solutions.')
    
    # Declare result arrays
    yghSolution = np.zeros((P, T, P, T, R))
    yijSolution = np.zeros((P, T, P, T, R))
    ySolution = np.zeros((P, T, P, T, R))
    xpSolution = np.zeros((P, T, P, T, R))
    xSolution = np.zeros((P,T,R))
    
    # Fill values
    for k in range(R):
        for i in range(P):
            for j in range(T):
                xSolution[i][j][k]=x[i][j][k].solution_value()
                for g in range(P):
                    for h in range(T):
                        yghSolution[i][j][g][h][k]=ygh[i][j][g][h][k].solution_value()
                        yijSolution[i][j][g][h][k]=yij[i][j][g][h][k].solution_value()
                        ySolution[i][j][g][h][k]=y[i][j][g][h][k].solution_value()
                        xpSolution[i][j][g][h][k]=xp[i][j][g][h][k].solution_value()
                        
    return z1.solution_value(), z2.solution_value(), yghSolution,yijSolution,ySolution,xpSolution,xSolution

In [3]:
def update_subgradient(ygh,yij,y,xp,E,surrogateDualBestZ3, surrogateDualBestZ4, lambda1, lambda2, dual_value_z3,dual_value_z4):
    
    # Define Surrogate Subgradient
    surrogateSubgradientZ3 = sum((ygh[i][j][g][h][k]-yij[i][j][g][h][k]-y[i][j][g][h][k]*E[i][j])
            for i in range(P)
            for j in range(T)
            for g in range(P)
            for h in range(T)
            for k in range(R)
    )

    surrogateSubgradientZ4 = sum((y[i][j][g][h][k]+y[g][h][i][j][k]-xp[g][h][i][j][k])
            for i in range(P)
            for j in range(T)
            for g in range(P)
            for h in range(T)
            for k in range(R)
    )
    print("z3",surrogateSubgradientZ3)
    print("z4",surrogateSubgradientZ4)
    # Calculate Step Size
    stepSizeZ3 = (surrogateDualBestZ3-dual_value_z3)/(surrogateSubgradientZ3)**2
    stepSizeZ4 = (surrogateDualBestZ4-dual_value_z4)/(surrogateSubgradientZ4)**2
    gap_tolerance = 0.001
    # New lambdas
    lambda1New = lambda1+stepSizeZ3*abs(surrogateSubgradientZ3)
    lambda2New = lambda2+stepSizeZ4*abs(surrogateSubgradientZ4)
    stopFlag = check_stop_criterion(lambda1,lambda2,lambda1New,lambda2New,gap_tolerance)
    return lambda1New, lambda2New, stopFlag

In [4]:
def calculate_dual_objective_value(z1, z2 ,ygh, yij, y, E, xp, Lambda1, Lambda2,surrogateDualBestZ3,surrogateDualBestZ4):
    dual_value_z3 = 0
    dual_value_z4 = 0
    for i in range(P):
        for j in range(T):
            for g in range(P):
                for h in range(T):
                    for k in range(R):
                        dual_value_z3 += Lambda1[i][j][g][h][k]*(ygh[i][j][g][h][k]-yij[i][j][g][h][k]-y[i][j][g][h][k]*E[i][j])
                        dual_value_z4 += Lambda2[i][j][g][h][k]*(y[i][j][g][h][k]+y[g][h][i][j][k]-xp[g][h][i][j][k])
    
    dual_value_z3 = dual_value_z3+z1+z2
    dual_value_z4 = dual_value_z4+z1+z2
    
    if dual_value_z3<surrogateDualBestZ3:
        surrogateDualBestZ3=dual_value_z3
    if dual_value_z4<surrogateDualBestZ4:
        surrogateDualBestZ4=dual_value_z4
    return dual_value_z3, dual_value_z4,surrogateDualBestZ3, surrogateDualBestZ4


In [5]:
def check_stop_criterion(lambda1,lambda2,lambda1New,lambda2New,gap_tolerance):
    if np.linalg.norm(lambda1New-lambda1) < gap_tolerance:
        if np.linalg.norm(lambda2New-lambda2) < gap_tolerance:
            return 1
        else:
            return 0

In [6]:
def heuristic_method(ygh, yij, y, E, xp):
    for i in range(P):
        for j in range(T):
            for g in range(P):
                for h in range(T):
                    for k in range(R):
                        if y[i][j][g][h][k]+y[g][h][i][j][k] == x[i][j][k]*x[g][h][k]:
                            if ygh[i][j][g][h][k]-yij[i][j][g][h][k]-y[i][j][g][h][k]*E[i][j]>=0:
                                continue
                            else:
                                x[i][j][k]=1-x[i][j][k]
                        else:
                            pass
    return feasible_solution

In [7]:
# Declare the MILP sovler with the sat backend
solver = pywraplp.Solver.CreateSolver('SAT')
if not solver:
    print("No such solver")

In [8]:
# Define Constant Variables

# Estimated Processing Time
E = [[60,90,50],[40,60,80],[50,70,60],[70,60,50],[45,55,65]]

# Transportation Time
L = [[0,30,30,60],[30,0,60,30],[30,60,0,30],[60,30,30,0]]

In [9]:
# Define Decision Variables

# Assume that there are 4 robots and 5 products need to be processed.
# Each product process contains 3 tasks.
R=4
P=5
T=3

# Start time of task ij
s = [[solver.IntVar(0, solver.infinity(), f's_{i}_{j}') for j in range(T)] for i in range(P)]

# Task allocation. Equals 1 if t_{ij} is performed on robot k, 0 otherwise.
x = [[[solver.IntVar(0,1,f'x_{i}_{j}_{k}') for k in range(R)] for j in range(T)] for i in range(P)]

# Task sequence. Equals 1 if x_{ij} is performed before x_{gh} on robot k, 0 otherwise.
y = [[[[[solver.IntVar(0,1,f'y_{i}_{j}_{g}_{h}_{k}') for k in range(R)] for h in range(T)] for g in range(P)] for j in range(T)] for i in range(P)]

print('Number of variables =', solver.NumVariables())

Number of variables = 975


In [10]:
# Define the Lagrangian Multipliers

Lambda1 = np.zeros((P, T, P, T, R))
Lambda2 = np.zeros((P, T, P, T, R))

In [11]:
# Add intermediate variables

# Two variables multiplication. xp = x[i][j][k]*x[g][h][k]
xp = [[[[[solver.IntVar(0,1,f'xp_{i}_{j}_{g}_{h}_{k}') for k in range(R)] for h in range(T)] for g in range(P)] for j in range(T)] for i in range(P)]
for k in range(R):
    for i in range(P):
        for j in range(T):
            for g in range(P):
                for h in range(T):
                    solver.Add(xp[i][j][g][h][k]<=x[i][j][k])
                    solver.Add(xp[i][j][g][h][k]<=x[g][h][k])
                    solver.Add(xp[i][j][g][h][k]>=x[i][j][k]+x[g][h][k]-1)

# Workload
v = [solver.IntVar(0,solver.infinity(),f'v_{k}') for k in range(R)]
c = [[[x[i][j][k]*E[i][j] for k in range(R)] for j in range(T)] for i in range(P)]
workLoad = [sum(sum([[c[i][j][k] for j in range(T)] for i in range(P)],[])) for k in range(R)]
workLoad_hat = sum(workLoad)/R
for k in range(R):
    solver.Add(v[k]>=workLoad[k]-workLoad_hat)
    solver.Add(v[k]>=workLoad_hat-workLoad[k])

# Robots of next task
f = [[[[solver.IntVar(0,1,f'f_{i}_{j}_{k}_{w}') for w in range(R)] for k in range(R)] for j in range(T-1)] for i in range(P)]
for i in range(P):
    for j in range(T-1):
        for k in range(R):
            for w in range(R):
                solver.Add(f[i][j][k][w]<=x[i][j][k])
                solver.Add(f[i][j][k][w]<=x[i][j+1][w])
                solver.Add(f[i][j][k][w]>=x[i][j][k]+x[i][j+1][w]-1)
                
# yij = y[i][j][g][h][k]*s[i][j]
# ygh = y[i][j][g][h][k]*s[g][h]
yij = [[[[[solver.IntVar(0,solver.infinity(),f'yij_{i}_{j}_{g}_{h}_{k}') for k in range(R)] for h in range(T)] for g in range(P)] for j in range(T)] for i in range(P)]
ygh = [[[[[solver.IntVar(0,solver.infinity(),f'ygh_{i}_{j}_{g}_{h}_{k}') for k in range(R)] for h in range(T)] for g in range(P)] for j in range(T)] for i in range(P)]
for k in range(R):
    for i in range(P):
        for j in range(T):
            for g in range(P):
                for h in range(T):
                    solver.Add(yij[i][j][g][h][k]<=100000*y[i][j][g][h][k])
                    solver.Add(yij[i][j][g][h][k]<=s[i][j])
                    solver.Add(yij[i][j][g][h][k]>=s[i][j]-(1-y[i][j][g][h][k])*100000)
                    solver.Add(yij[i][j][g][h][k]>=0)
                    
                    solver.Add(ygh[i][j][g][h][k]<=100000*y[i][j][g][h][k])
                    solver.Add(ygh[i][j][g][h][k]<=s[g][h])
                    solver.Add(ygh[i][j][g][h][k]>=s[g][h]-(1-y[i][j][g][h][k])*100000)
                    solver.Add(ygh[i][j][g][h][k]>=0) 

In [12]:
# Define constraints

# One task could be only processed on one robot
for i in range(P):
    for j in range(T):
            solver.Add(sum([x[i][j][k]*1 for k in range(R)])==1)

# Every Start time should bigger than zero
for i in range(P):
    for j in range(T):
        solver.Add(s[i][j]>=0)
        
# The order between two tasks of one product
routeLength = [[[[f[i][j][k][w]*L[k][w] for w in range(R)] for k in range(R)] for j in range(T-1)] for i in range(P)]
for i in range(P):
    for j in range(T-1):
        solver.Add(s[i][j+1]-(s[i][j]+E[i][j]+sum(sum(routeLength[i][j],[])))>=0)
            
# Completion condition
for i in range(P):
    solver.Add(sum(sum([[x[i][j][k]*1 for k in range(R)] for j in range(T)],[]))==3)
    
# Must at least one robot could run
for k in range(R):
    solver.Add(20-sum(sum([[x[i][j][k]*1 for j in range(T)] for i in range(P)],[]))>=0)


In [13]:
# Define Objective functions

# Minimize the makespan
makespan = solver.IntVar(0, solver.infinity(), f'm')
for i in range(P):
    for j in range(T):
        solver.Add(makespan>=s[i][j]+E[i][j])
#solver.Minimize(cycleTime)
z1 = makespan

# Minimize the variance of workload
#solver.Minimize(1/R*sum(y))
z2 = sum(v)/R

# Weights of each objective functions
weights = [(1, 0), (0.9, 0.1), (0.8, 0.2), (0.7, 0.3), (0.6, 0.4), (0.5, 0.5), (0.4, 0.6), (0.3, 0.7), (0.2, 0.8), (0.1, 0.9)]

z = weights[5][0] * z1 + weights[5][1] *z2

In [14]:
# Current Best Surrogate dual
surrogateDualBestZ3 = 150
surrogateDualBestZ4 = 150

In [15]:
for i in range(100):
    z1CS,z2CS,yghCS,yijCS,yCS,xpCS,xCS=solveSubproblem(Lambda1, Lambda2,z)
    dual_value_z3, dual_value_z4, surrogateDualBestZ3, surrogateDualBestZ4 = calculate_dual_objective_value(z1CS, z2CS ,yghCS, yijCS, yCS, E, xpCS, Lambda1, Lambda2,surrogateDualBestZ3,surrogateDualBestZ4)
    Lambda1, Lambda2, stopFlag = update_subgradient(yghCS, yijCS, yCS, xpCS, E,surrogateDualBestZ3, surrogateDualBestZ4, Lambda1, Lambda2, dual_value_z3,dual_value_z4)
    if stopFlag == 1:
        break